---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
# path = "./../Datasets/"
path=""

In [ ]:
def cleanUniversityTownTxt(row):
    region_name = ""
    state_name =  ""
    if row.str.contains('edit')[0] == True:
        state_name = row.str.replace("\[edit\]","")[0]
#         row = row.str.replace("\[edit\]","")[0]
    else:
        try:
            region_name = row.str.replace(r"\(.*","").str.strip()[0]
#             row = row.str.replace(r"\(.*","").str.strip()[0]
        except:
            print("ERROR : "+row)
    if region_name != "":
        row["RegionName"] = region_name
    
    if state_name != "":
        row["State"] = state_name
    return row

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    uni_towns = pd.read_fwf(path+"university_towns.txt",header = None)
    
    uni_towns = uni_towns.apply(cleanUniversityTownTxt,axis = 1)
    
    uni_towns.loc[:,"State"].fillna(method = 'ffill', inplace = True)
    uni_towns = (uni_towns.dropna(subset=["RegionName"])
                          .drop([0],axis = 1))
    return uni_towns

In [ ]:

GDP_lev = pd.read_excel(path+"gdplev.xls", skiprows=2, header=[3],index_col=None)

GDP_lev = (GDP_lev
#                   
                  .drop(['Unnamed: 0', 'GDP in billions of current dollars',
                         'GDP in billions of chained 2009 dollars', 'Unnamed: 3','Unnamed: 7'],axis = 1)
                  .dropna(how="all")
                  .reset_index()
                  .drop(["index"],axis = 1)
                  .rename(columns = {"Unnamed: 4" : "Year-Quarter","GDP in billions of current dollars.1":"GDP in billions of current dollars","GDP in billions of chained 2009 dollars.1":"GDP in billions of chained 2009 dollars"})
           )

# GDP_lev.drop(["Annual"],axis = 1,inplace= True)
# GDP_lev.reset_index(inplace= True)
# GDP_lev.drop(["index"],axis = 1,inplace= True)
# GDP_lev.drop(["GDP in billions of chained 2009 dollars.1"],level=1,axis = 1,inplace= True)
# # GDP_lev.rename({"Unnamed: 3_level_1" : "Year-Quarter"},axis=1,inplace= True)
# GDP_lev.dropna(how="all", inplace= True)
# GDP_lev.columns =GDP_lev.columns.droplevel()
# GDP_lev["Year-Quarter"] = GDP_lev["Unnamed: 3_level_1"]
# GDP_lev.drop(["Unnamed: 3_level_1"],axis = 1,inplace= True)


GDP_lev = GDP_lev[GDP_lev["Year-Quarter"] >= "2000q1"]
# print(GDP_lev.test)

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    answer = GDP_lev[(GDP_lev["GDP in billions of current dollars"] > GDP_lev["GDP in billions of current dollars"].shift(1)) 
        &(GDP_lev["GDP in billions of current dollars"].shift(-1) < GDP_lev["GDP in billions of current dollars"]) 
        &(GDP_lev["GDP in billions of current dollars"].shift(-2) < GDP_lev["GDP in billions of current dollars"].shift(-1))]["Year-Quarter"].iloc[0]
    
    return answer

In [ ]:
get_recession_start()

In [ ]:
GDP_lev[(GDP_lev["Year-Quarter"]>="2008q1")&(GDP_lev["Year-Quarter"]<="2010q1")]

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    answer = GDP_lev[(GDP_lev["Year-Quarter"] > get_recession_start()) 
        &(GDP_lev["GDP in billions of current dollars"] > GDP_lev["GDP in billions of current dollars"].shift(1)) 
        &(GDP_lev["GDP in billions of current dollars"].shift(2) < GDP_lev["GDP in billions of current dollars"].shift(1))]["Year-Quarter"].iloc[0]
    return answer

In [ ]:
get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    answer1 =  GDP_lev[(GDP_lev["Year-Quarter"] >= get_recession_start()) &(GDP_lev["Year-Quarter"] <= get_recession_end())]["GDP in billions of current dollars"].min()
    answer = GDP_lev[GDP_lev["GDP in billions of current dollars"] == answer1]["Year-Quarter"].iloc[0]
    return answer

In [ ]:
get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv(path+"City_Zhvi_AllHomes.csv")
    
    columns_to_drop = []
    for col in housing_data.columns:
        if "-" in col:
            year,month = int(col.split("-")[0]),int(col.split("-")[1])
            if year < 2000 :
                columns_to_drop.append(col)    
    housing_data.drop(columns_to_drop,axis=1,inplace=True)
    
    housing_data["State"].replace(states,inplace=True)
    housing_data.set_index(["State","RegionName"],inplace=True)
    
    housing_data = housing_data.iloc[:,4:]
    
    quaters_cal = []
    data_size = len(housing_data.columns)
    for col in housing_data.columns:
            year,month = col.split("-")[0],col.split("-")[1]
            if int(month) in [1,4,7,10]:
               
                if housing_data.columns.get_loc(col) < data_size:
                    quaters_cal.append(housing_data.columns.get_loc(col))
                if housing_data.columns.get_loc(col)+1 < data_size:
                    quaters_cal.append(housing_data.columns.get_loc(col)+1)
                if housing_data.columns.get_loc(col)+2 < data_size:
                    quaters_cal.append(housing_data.columns.get_loc(col)+2)    
                meanSeries = housing_data.iloc[:,[housing_data.columns.get_loc(col),housing_data.columns.get_loc(col)+1,housing_data.columns.get_loc(col)+2]].mean(axis=1)

                if int(month)== 1:
        #             if year == "2000":
                    housing_data[year+"q1"] =meanSeries
                if int(month)== 4:
                    housing_data[year+"q2"] =meanSeries
                if int(month)== 7:
                    housing_data[year+"q3"] =meanSeries
                if int(month)== 10:
                    housing_data[year+"q4"] =meanSeries    

    housing_data.drop(housing_data.columns[quaters_cal],axis=1,inplace= True)
    return housing_data

In [ ]:
data = convert_housing_data_to_quarters()
data


In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    res_start = get_recession_start()
    res_bottom = get_recession_bottom()
    res_end = get_recession_end()
    
    print("Recession start = {} \n Recession Bottom = {} \n Recession end = {}".format(res_start,res_bottom,res_end))
    
    uni_towns = get_list_of_university_towns()
    
    GDP_data = GDP_lev
    
    house_quaters = convert_housing_data_to_quarters()
    
    list_state_town = list(zip(uni_towns["State"],uni_towns["RegionName"]))
    uni_town_data = house_quaters.loc[house_quaters.index.isin(list_state_town),[res_start,res_bottom]]
    non_uni_town_data = house_quaters.loc[~house_quaters.index.isin(list_state_town),[res_start,res_bottom]]
    
    uni_town_data["priceRatio"] = uni_town_data.iloc[:,0]/uni_town_data.iloc[:,1]
    non_uni_town_data["priceRatio"] = non_uni_town_data.iloc[:,0]/non_uni_town_data.iloc[:,1]
    statistic,pvalue = ttest_ind(uni_town_data["priceRatio"],non_uni_town_data["priceRatio"], nan_policy='omit')
    
    different = False
    if pvalue < 0.01:
        different = True
    else:
        different = False
    
    better=""
    if uni_town_data["priceRatio"].mean() < non_uni_town_data["priceRatio"].mean():
        better = "university town"
    else:
        better = "non-university town"
    
    return (different, pvalue, better)

In [ ]:
answer = run_ttest()
answer